In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy import stats
from scipy import sparse
import scipy
from collections import defaultdict
import implicit
from implicit.als import AlternatingLeastSquares
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVDpp
import rectorch
from rectorch.models.baseline import SLIM, Random, Popularity
from rectorch.data import DataProcessing
from rectorch.samplers import ArrayDummySampler, SparseDummySampler
from rectorch.evaluation import evaluate
from rectorch.utils import collect_results, prepare_for_prediction

# Data Parsing

In [2]:
# initialize data
item_threshold = 1 # used to filter out user/artist pairs that have been 
                   #listened to less than the threshold number of times
popular_artist_fraction = 0.2 # top cutoff for what we consider popular artists, in this case the top 20%

user_events_file = 'data/user_events.txt'
low_user_file = 'data/low_main_users.txt'
medium_user_file = 'data/medium_main_users.txt'
high_user_file = 'data/high_main_users.txt'

In [3]:
#read in user events file
cols = ['user', 'artist', 'album', 'track', 'timestamp']
df_events = pd.read_csv(user_events_file, sep='\t', names=cols)
print('No. of user events: ' + str(len(df_events)))
df_events.head() # check it is all read in properly

No. of user events: 28718087


,user,artist,album,track,timestamp
0,31435741,2,4,4,1385212958
1,31435741,2,4,4,1385212642
2,31435741,2,4,4,1385212325
3,31435741,2,4,4,1385209508
4,31435741,2,4,4,1385209191


## User-Artist Matrix

In [4]:
# create unique user-artist matrix
df_events = df_events.groupby(['user', 'artist']).size().reset_index(name='listens')
print('No. user-artist pairs: ' + str(len(df_events)))
# each row contains a unique user-artist pair, along with how many times the
# user has listened to the artist
#df_events.sort_values(by=['user'])

No. user-artist pairs: 1755361


In [5]:
df_events.head(1400)

,user,artist,listens
0,1021445,12,43
1,1021445,16,1
2,1021445,28,7
3,1021445,29,1
4,1021445,46,1
...,...,...,...
1395,1021445,1864220,3
1396,1021445,1864221,4
1397,1021445,1864222,1
1398,1045479,3,9


In [6]:
# filters out artist/user pairs who havent been listened two more than
# item_threshold amount of times to reduce potential load
# kept to 1 currently, so we dont filter out any data
df_events = df_events[df_events['listens'] >= item_threshold] 

# With 1, we see no difference between user-artist pairs here
print('No. filtered user-artist pairs: ' + str(len(df_events))) 

# here, we see the number of unique artists in our matrix
print('No. unique artists: ' + str(len(df_events['artist'].unique())))

No. filtered user-artist pairs: 1755361
No. unique artists: 352805


#### How many artists have users listened to?

In [7]:
# get matrix where each row is a user-id and how many artists they've 
#listened to
user_dist = df_events['user'].value_counts() 

# counts how many unique users there are. prints out user id & a count of how 
# many rows they're included in, which effectively shows how many artists 
# they listen to
num_users = len(user_dist)
print('Mean artists of all users: ' + str(user_dist.mean()))
print('Min artists of all users: ' + str(user_dist.min()))
print('Max artists of all users: ' + str(user_dist.max()))

user_dist.head()

Mean artists of all users: 585.1203333333333
Min artists of all users: 18
Max artists of all users: 4011


41888522    4011
4393555     3700
40029632    3678
26874346    3544
29736410    3529
Name: user, dtype: int64

#### How many users listen to an artist?

In [8]:
# get artist distribution
# same as previous but with artists, shows artist-id and how many times they
# were listened to buy unique users
artist_dist = df_events['artist'].value_counts()
num_artists = len(artist_dist)
print('No. artists: ' + str(num_artists))
df_events['artist'].value_counts().head

No. artists: 352805


<bound method NDFrame.head of 135        1389
1602       1359
46         1325
320        1297
27         1290
           ... 
3124286       1
1029181       1
1023032       1
1008679       1
3087545       1
Name: artist, Length: 352805, dtype: int64>

In [9]:
# get number of  popular artists
num_top_artists = int(popular_artist_fraction * num_artists)

# getting the top top_fraction (0.2) percent of artists, so finding how many
# artists make up 20% of total artists, and then only using the artists those
#number of the most popular aritsts
top_artist_dist = artist_dist[:num_top_artists]
print('No. top artists: ' + str(len(top_artist_dist)))

No. top artists: 70561


In [10]:
# read in users
# user file is just user_id and their mainstreaminess value 
low_users = pd.read_csv(low_user_file, sep=',').set_index('user_id')
medium_users = pd.read_csv(medium_user_file, sep=',').set_index('user_id')
high_users = pd.read_csv(high_user_file, sep=',').set_index('user_id')
num_users = len(low_users) + len(medium_users) + len(high_users)
print('Num users: ' + str(num_users))

Num users: 3000


## Calculating GAP of User Profiles

In [11]:
# placeholder vars for numerator of GAPp, waiting to be divided by sie of group
low_gap_p = 0
medium_gap_p = 0
high_gap_p = 0
total_gap_p = 0
#Count for sanity check
low_count = 0
med_count = 0
high_count = 0

for u, df in df_events.groupby('user'):
    
    no_user_artists = len(set(df['artist'])) # profile size //number of artists in users profile
    # get popularity (= fraction of users interacted with item) of user items and calculate average of it
    user_pop_artist_fraq = sum(artist_dist[df['artist']] / num_users) / no_user_artists 
    
    if u in low_users.index: # get user group-specific values
        low_gap_p += user_pop_artist_fraq
        low_count += 1
    elif u in medium_users.index:
        medium_gap_p += user_pop_artist_fraq
        med_count += 1
    else:
        high_gap_p += user_pop_artist_fraq
        high_count += 1

total_gap_p = (low_gap_p + medium_gap_p + high_gap_p) / num_users
low_gap_p /= len(low_users) # average popularity of items/artists in low/med/high groups (gap = group average popularity)
medium_gap_p /= len(medium_users)
high_gap_p /= len(high_users)
print('Low count (for check): ' + str(low_count))
print('Med count (for check): ' + str(med_count))
print('High count (for check): ' + str(high_count))
print(low_gap_p)
print(medium_gap_p)
print(high_gap_p)

Low count (for check): 1000
Med count (for check): 1000
High count (for check): 1000
0.04963328792099549
0.054371119359489226
0.06286028679778642


### Min-Max Scaling Ratings

In [ ]:
### Scale listening counts on a scale from 1-1000
scaled_df_events = pd.DataFrame()
for user_id, group in df_events.groupby('user'):
    #print(group)
    min_listens = group['listens'].min()
    max_listens = group['listens'].max()
    std = (group['listens'] - min_listens) / (max_listens - min_listens)
    scaled_listens = std * 999 + 1
    to_replace = group.copy()
    to_replace['listens'] = scaled_listens
    #print(to_replace)
    scaled_df_events = scaled_df_events.append(to_replace)
scaled_df_events.head()   
#df_events.groupby('user').head()

# Analysis of Cutting Edge RecSys Algorithms

In [ ]:
#Normalize data to compare artist popularity vs recommendations
#Normalize artist popularity
normalized_artist_dist = pd.DataFrame(artist_dist)
normalized_artist_dist.columns = ['listens']
normalized_artist_dist['listens'] /= num_users
normalized_artist_dist.head()

num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0
#num_times_recommended.head()
#normalized_artist_dist.head()

In [ ]:
def user_events_file_to_lil_matrix():
    # Artist to User matrix where artist_user_matrix[a, u] = num of times user u listened to artist a

    # 352805, 3000 (total artists, users)
    rows, cols = 352805, 3000
    artist_user_matrix = scipy.sparse.lil_matrix((rows, cols), dtype=int)

    # user	artist	album	track	timestamp

    user_dict = {}  # simplify user id to 1, 2, 3 ...
    artist_dict = {}

    # populate with user_events_file
    with open("/home/jimi/music_fairness/LFM-1b-surprise-and-implicit-analysis/ML_Base_Project/data/user_events.txt", 'r') as fp:
        line = fp.readline()
        loop_count = 0
        while line:
            # get data from line
            line = fp.readline()
            parts = line.split("\t")

            # end case
            try:
                user_id = int(parts[0])
                artist_id = int(parts[1])
            except ValueError:
                print("end of file " + line)
                break

            # use user_dict to shorten user_id
            if user_id not in user_dict:
                # this user_id has not bee seen
                user_dict[user_id] = len(user_dict)
            user_idx = user_dict[user_id]

            # use track_dict to shorten track_id
            if artist_id not in artist_dict:
                # this user_id has not bee seen
                artist_dict[artist_id] = len(artist_dict)
            artist_idx = artist_dict[artist_id]

            # increment count of user to track
            artist_user_matrix[artist_idx, user_idx] += 1

            # progress marker
            loop_count = loop_count + 1
            if loop_count % 10000000 == 0:
                print(str(loop_count) + "/ 28718087")  # / num of lines in file

    print(len(user_dict))
    print(len(artist_dict))

    # helpful dicts for converting artist and user count back to their ids
    user_count_to_id_dict = {v: k for k, v in user_dict.items()}
    artist_count_to_id_dict = {v: k for k, v in artist_dict.items()}

    return artist_user_matrix, user_dict, artist_dict, user_count_to_id_dict, artist_count_to_id_dict


## FUNCTIONS FROM from jmsteinw's blog post ## a_u_tuple


In [ ]:
def make_train(ratings, pct_test=0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings,
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix.

    parameters:

    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix.

    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the
    training set for later comparison to the test set, which contains all of the original ratings.

    returns:

    training_set - The altered version of the original data with a certain percentage of the user-item pairs
    that originally had interaction set back to zero.

    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order
    compares with the actual interactions.

    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy()  # Make a copy of the original set to be the test set.
    test_set[test_set != 0] = 1  # Store the test set as a binary preference matrix
    training_set = ratings.copy()  # Make a copy of the original data we can alter as our training set.
    nonzero_inds = training_set.nonzero()  # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))  # Zip these pairs together of user,item index into list
    random.seed(0)  # Set the random seed to zero for reproducibility
    num_samples = int(
        np.ceil(pct_test * len(nonzero_pairs)))  # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples)  # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples]  # Get the user row indices
    item_inds = [index[1] for index in samples]  # Get the item column indices
    training_set[user_inds, item_inds] = 0  # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros()  # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds))  # Output the unique list of user rows that were altered


# Usage
# a_to_u_train, a_to_u_test, altered_users = make_train(csr_sparse_matrix, pct_test = 0.2)

In [ ]:
a_u_tuple = user_events_file_to_lil_matrix()
a_u_matrix = a_u_tuple[0]
back_to_user_id = a_u_tuple[3] #dictionary to convert smaller user ids to original
back_to_artist_id = a_u_tuple[4]# same above but for artists
print("done")

In [ ]:
u_to_a_train, u_to_a_test, altered_users = make_train(a_u_matrix.T.tocsr(), pct_test=0.2) #makes training/test set
print("done")

In [ ]:
# split original matrix into user vector and artist vector through ALS
user_vecs, artists_vecs = implicit.alternating_least_squares((u_to_a_train).astype('double'),iterations=50, factors=50)
print("done")

In [ ]:
print(np.array(user_vecs[0,:].dot(artists_vecs.T))[:10]) # gets first 5 recommendations

In [ ]:
def get_top_artists_v2(user_vecs, artists_vecs, mf_train, back_to_user_id,back_to_artist_id, num=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        user_vecs: the feature vector for users
        mf_train: training matrix
        artists_vecs: the feature vector for artists
        back_to_user_id: dictionary to convert back to original user id
        back_to_artist_id: dictionary to convert back to original artist id
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id), ...] of size n.
    """
    top_artists_dict = defaultdict(list)
        # creating dictionary where user id is the key, and the val is a list of tuples of the artist id and 
        # the rating it thinks the user would give it
        
    for user_id in range(len(user_vecs)):
        
        pref_vec = mf_train[user_id,:].toarray() # Get the ratings from the training set ratings matrix
        pref_vec = pref_vec.reshape(-1)
        pref_vec =  pref_vec + 1 # Add 1 to everything, so that artists not listened to yet are equal to 1
        pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
        
        rec_vector = user_vecs[user_id,:].dot(artists_vecs.T) # Get dot product of user vector and all artist vectors
        # Scale this recommendation vector between 0 and 1
        
        min_max = MinMaxScaler()
        rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
        recommend_vector = pref_vec*rec_vector_scaled 
        # Items already purchased have their recommendation multiplied by zero
        
        product_idx = np.argsort(recommend_vector)[::-1][:num] # Sort the indices of the items into order 
        # of best recommendations
        rec_list = [] # start empty list to store items
        actual_user_id = back_to_user_id[user_id]
        #print(actual_user_id)
        
        for index in product_idx:
            actual_artist_id = back_to_artist_id[index]
            rec_list.append(actual_artist_id)
            
        top_artists_dict[actual_user_id] = rec_list
    print("done")
    return top_artists_dict
    
    

In [ ]:
test_dict = get_top_artists_v2(user_vecs, artists_vecs, u_to_a_train,back_to_user_id, back_to_artist_id,num = 10) 
#gets top n artists for all the users in the dataset
print("done")

In [ ]:
print(test_dict[6532902]) #testing out getting top recommendations for a user

In [ ]:
print(artist_dist[6443]) #getting popularity of artist by indexing by id

In [ ]:
artist_dist.head()

In [ ]:
# Train and test the four algorithms on our data set
low_gap_r_list = []
medium_gap_r_list = []
high_gap_r_list = []
total_gap_r_list= []
#keeps track of the GAPr of each of the algs

alg_recommendations = [] #Keeps track of how many times each artist was recommended

#Will put for loop here with training, but for now we already have predictions
    
num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0 #resets for each algorithm
    
    
# resets to 0 before calculating for new alg
low_gap_r = 0
medium_gap_r = 0
high_gap_r = 0
    
#keeps track of group size since it is unpredictable since the test set is selected randomly
num_low_users = 0
num_medium_users = 0
num_high_users = 0
    
#keeps track of the mean absolute error of the current alg
low_mae = 0
medium_mae = 0
high_mae = 0
top_artists = test_dict #Gets the top N recommendations for each user
    

for user_id, ratings in top_artists.items():
    artist_id_list = [] #user profile size to compute top fraction in GAPr
    for artist_id in ratings:
            
        artist_id_list.append(artist_id)
        num_times_recommended.loc[artist_id] += 1 #increments the number of times the artist was recommended

    gap = sum(artist_dist[artist_id_list]/ num_users) / len(artist_id_list) #fraction in numerator for GAPr
    #print("gap:" ,gap)
    if user_id in low_users.index: #summation of all of the top fractions to compute numerator for GAPr
        low_gap_r += gap
        num_low_users += 1 #increments group size to get the denominator to compute GAPr

    elif user_id in medium_users.index:
        medium_gap_r += gap
        num_medium_users += 1

    elif user_id in high_users.index:
        high_gap_r += gap
        num_high_users += 1
alg_recommendations.append(num_times_recommended)
            
total_gap_r = (low_gap_r + medium_gap_r + high_gap_r) / (num_low_users + num_medium_users + num_high_users)
low_gap_r = low_gap_r / num_low_users #Computing GAPr for each group size for each algorithm
medium_gap_r = medium_gap_r / num_medium_users
high_gap_r = high_gap_r / num_high_users

total_gap_r_list.append(total_gap_r)
low_gap_r_list.append(low_gap_r)
medium_gap_r_list.append(medium_gap_r)
high_gap_r_list.append(high_gap_r)

print("done")

In [ ]:
#Computes change in GAP for each RecSys alg
delta_gap_low_list = []
delta_gap_medium_list = []
delta_gap_high_list = []
delta_gap_total_list = []

for i in range(len(low_gap_r_list)):
    delta_gap_low = ((low_gap_r_list[i] - low_gap_p) / low_gap_p)
    delta_gap_medium = ((medium_gap_r_list[i] - medium_gap_p) / medium_gap_p)
    delta_gap_high = ((high_gap_r_list[i] - high_gap_p) / high_gap_p)
    delta_gap_total = ((total_gap_r_list[i] - total_gap_p) / total_gap_p)
    
    delta_gap_low_list.append(delta_gap_low)
    delta_gap_medium_list.append(delta_gap_medium)
    delta_gap_high_list.append(delta_gap_high)
    delta_gap_total_list.append(delta_gap_total)
    
print(delta_gap_low_list[0])

print(delta_gap_medium_list[0])

print(delta_gap_high_list[0])

print(delta_gap_total_list[0])


In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics.

    parameters:

    - predictions: your prediction output

    - test: the actual target result you are comparing to

    returns:

    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    # shuffle list of predictions (shuffle function)
    # shuffling dissassociates link to artist - > roc of .5 (random)
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered.

    parameters:

    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model

    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one.

    altered_users - The indices of the users where at least one user/item pair was altered from make_train function

    test_set - The test set constucted earlier from make_train function

    returns:

    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''

    store_auc = []  # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = []  # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis=0)).reshape(-1)  # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users:  # Iterate through each user that had an item altered
        training_row = training_set[user, :].toarray().reshape(-1)  # Get the training set row
        zero_inds = np.where(training_row == 0)  # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user, :]
        pred = user_vec.dot(item_vecs).toarray()[0, zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user, :].toarray()[0, zero_inds].reshape(-1)
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training
        pop = pop_items[zero_inds]  # Get the item popularity for our chosen items
        curr_auc_score = auc_score(pred, actual)
        store_auc.append(curr_auc_score)  # Calculate AUC for the given user and store
        curr_pop_score = auc_score(pop, actual)
        popularity_auc.append(curr_pop_score)  # Calculate AUC using most popular and score
        # print(user, "\t", curr_auc_score , "\t", curr_pop_score)
    # End users iteration

    return float('%.3f' % np.mean(store_auc)), float('%.3f' % np.mean(popularity_auc))
    # Return the mean AUC rounded to three decimal places for both test and popularity benchmark


In [ ]:
rec_auc, pop_auc = calc_mean_auc(u_to_a_train, altered_users,[sparse.csr_matrix(user_vecs), sparse.csr_matrix(artists_vecs.T)],
                                             u_to_a_test)
print(rec_auc)
print(pop_auc)

## Getting Users From Each Popularity Group & Their 10 Most Listened To Artists

### (For Spotify Profiles)

In [ ]:
toList = df_events.loc[df_events['user'] == 42845367].sort_values(by=['listens'], ascending=False)
toList.head() #grabbing random users top 10 artists in 1 of the 3 groups

In [ ]:
to_list_2 = toList['artist'].tolist()[:20]
print(to_list_2)

# Rectorch Testing

In [35]:
data_events = df_events.copy(deep=True)
data_events.to_csv('data_events.csv',index=False,header=False )
print("done")

done


In [36]:
!head data_events.csv

1021445,12,43
1021445,16,1
1021445,28,7
1021445,29,1
1021445,46,1
1021445,57,29
1021445,73,41
1021445,74,4
1021445,105,4
1021445,113,1


In [12]:
cfg_data = {
    "processing": {
        "data_path": "data_events.csv",
        "threshold": 0,
        "separator": ",",
        "header": None,
        "u_min": 50,
        "i_min": 50
    },
    "splitting": {
        "split_type": "horizontal",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 0.1,
        "test_size": 0.1,
        "test_prop": 0.2
    }
}

cfg_data_full = {
    "processing": {
        "data_path": "data_events.csv",
        "threshold": 0,
        "separator": ",",
        "header": None,
        "u_min": 0,
        "i_min": 0
    },
    "splitting": {
        "split_type": "horizontal",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 0.1,
        "test_size": 0.1,
        "test_prop": 0.2
    }
}

In [13]:
dataset = DataProcessing(cfg_data).process_and_split()
dataset

[17:52:04-130421]  Reading raw data file data_events.csv.
[17:52:09-130421]  Applying filtering.
[17:52:09-130421]  Filtered 988317 ratings.
[17:52:09-130421]  Shuffling data.
[17:52:09-130421]  Creating training, validation and test set.


Dataset(n_users=2804, n_items=5131, n_ratings=767044)

In [43]:
dataset.test_set

,uid,iid,rating
427,0,2973,1
637,0,2360,17
315,0,358,1
12,0,1586,19
183,0,353,1
...,...,...,...
1754676,2803,1351,13
1754712,2803,1348,1
1754535,2803,176,1
1754969,2803,4178,1


In [14]:
sparse_sampler = SparseDummySampler(dataset, mode="train")

In [46]:
lambda_vals =  np.array([0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3])
beta_vals =  np.array([0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3])


error = np.zeros([lambda_vals.size,beta_vals.size])

sparse_sampler = SparseDummySampler(dataset, mode="train")
for i in range(len(lambda_vals)):
    for j in range(len(beta_vals)):
        sparse_sampler.train()
        
        slim = SLIM(l1_reg=lambda_vals[i], l2_reg=beta_vals[j])
        slim.train(sparse_sampler)
        
        sparse_sampler.test()
        
        results = evaluate(slim, sparse_sampler, ["ap@5000"])
        error[i,j] = collect_results(results).get("ap@5000")[0]
        print(collect_results(results))
        print(collect_results(results).get("ap@5000")[0])
        
print(error)
        
minError = np.argwhere(error == np.amax(error))
Lambda = lambda_vals[(minError[0][0])]
beta = beta_vals[(minError[0][1])]
print(Lambda)
print(beta)


[12:19:59-090421]  | item 513/5131 | ms/user 14.73 |
[12:20:06-090421]  | item 1026/5131 | ms/user 14.50 |
[12:20:13-090421]  | item 1539/5131 | ms/user 14.36 |
[12:20:21-090421]  | item 2052/5131 | ms/user 13.94 |
[12:20:28-090421]  | item 2565/5131 | ms/user 13.64 |
[12:20:35-090421]  | item 3078/5131 | ms/user 13.65 |
[12:20:41-090421]  | item 3591/5131 | ms/user 13.53 |
[12:20:48-090421]  | item 4104/5131 | ms/user 13.41 |
[12:20:55-090421]  | item 4617/5131 | ms/user 13.35 |
[12:21:02-090421]  | item 5130/5131 | ms/user 13.19 |
[12:21:02-090421]  | training complete | total training time 71.00 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.12851225090805365, 0.0801624333389584)}
0.12851225090805365


[12:21:13-090421]  | item 513/5131 | ms/user 14.72 |
[12:21:20-090421]  | item 1026/5131 | ms/user 14.43 |
[12:21:27-090421]  | item 1539/5131 | ms/user 14.14 |
[12:21:34-090421]  | item 2052/5131 | ms/user 13.84 |
[12:21:41-090421]  | item 2565/5131 | ms/user 13.51 |
[12:21:48-090421]  | item 3078/5131 | ms/user 13.60 |
[12:21:55-090421]  | item 3591/5131 | ms/user 13.26 |
[12:22:02-090421]  | item 4104/5131 | ms/user 13.17 |
[12:22:09-090421]  | item 4617/5131 | ms/user 13.27 |
[12:22:15-090421]  | item 5130/5131 | ms/user 13.12 |
[12:22:16-090421]  | training complete | total training time 70.37 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.12882273749332865, 0.0800273683108513)}
0.12882273749332865


[12:22:26-090421]  | item 513/5131 | ms/user 14.73 |
[12:22:33-090421]  | item 1026/5131 | ms/user 14.39 |
[12:22:41-090421]  | item 1539/5131 | ms/user 13.81 |
[12:22:48-090421]  | item 2052/5131 | ms/user 13.52 |
[12:22:54-090421]  | item 2565/5131 | ms/user 13.16 |
[12:23:01-090421]  | item 3078/5131 | ms/user 13.21 |
[12:23:08-090421]  | item 3591/5131 | ms/user 13.06 |
[12:23:14-090421]  | item 4104/5131 | ms/user 12.91 |
[12:23:21-090421]  | item 4617/5131 | ms/user 12.96 |
[12:23:28-090421]  | item 5130/5131 | ms/user 12.73 |
[12:23:28-090421]  | training complete | total training time 69.05 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


...

{'ap@5000': (0.0, 0.0)}
0.0


[13:11:55-090421]  | item 513/5131 | ms/user 3.31 |
[13:11:57-090421]  | item 1026/5131 | ms/user 3.29 |
[13:11:59-090421]  | item 1539/5131 | ms/user 3.29 |
[13:12:01-090421]  | item 2052/5131 | ms/user 3.29 |
[13:12:02-090421]  | item 2565/5131 | ms/user 3.29 |
[13:12:04-090421]  | item 3078/5131 | ms/user 3.29 |
[13:12:06-090421]  | item 3591/5131 | ms/user 3.29 |
[13:12:07-090421]  | item 4104/5131 | ms/user 3.29 |
[13:12:09-090421]  | item 4617/5131 | ms/user 3.30 |
[13:12:11-090421]  | item 5130/5131 | ms/user 3.29 |
[13:12:11-090421]  | training complete | total training time 16.92 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.0, 0.0)}
0.0


[13:12:13-090421]  | item 513/5131 | ms/user 3.36 |
[13:12:15-090421]  | item 1026/5131 | ms/user 3.34 |
[13:12:16-090421]  | item 1539/5131 | ms/user 3.34 |
[13:12:18-090421]  | item 2052/5131 | ms/user 3.33 |
[13:12:20-090421]  | item 2565/5131 | ms/user 3.34 |
[13:12:21-090421]  | item 3078/5131 | ms/user 3.34 |
[13:12:23-090421]  | item 3591/5131 | ms/user 3.34 |
[13:12:25-090421]  | item 4104/5131 | ms/user 3.34 |
[13:12:27-090421]  | item 4617/5131 | ms/user 3.34 |
[13:12:28-090421]  | item 5130/5131 | ms/user 3.35 |
[13:12:28-090421]  | training complete | total training time 17.17 s |
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.0, 0.0)}
0.0
[[0.12851225 0.12882274 0.12988474 0.13238474 0.13731052 0.1409856
  0.13895127 0.13062076]
 [0.13091648 0.13111136 0.13198311 0.13424593 0.13834158 0.14131486
  0.13877306 0.1303479 ]
 [0.13537905 0.1356008  0.13626014 0.13759539 0.13994234 0.14052816
  0.13709833 0.12844514]
 [0.13178956 0.13176923 0.1318709  0.13185112 0.13179571 0.13079366
  0.12743128 0.11991232]
 [0.10004898 0.10006405 0.10001021 0.09991465 0.09985705 0.09944694
  0.09835089 0.09482609]
 [0.06335927 0.06332919 0.06334652 0.06346481 0.06355087 0.06365465
  0.06411409 0.06393328]
 [0.02397401 0.02395416 0.02382432 0.02411175 0.02396452 0.02405855
  0.02473613 0.02574048]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
0.0003
0.03


In [47]:
#Learned best Lambda = 0.0003 and best Beta = 0.03
sparse_sampler.train()
slim = SLIM(l1_reg=Lambda, l2_reg=beta)
slim.train(sparse_sampler)

[13:26:59-090421]  | item 513/5131 | ms/user 13.48 |
[13:27:06-090421]  | item 1026/5131 | ms/user 13.20 |
[13:27:13-090421]  | item 1539/5131 | ms/user 13.07 |
[13:27:19-090421]  | item 2052/5131 | ms/user 12.69 |
[13:27:25-090421]  | item 2565/5131 | ms/user 12.41 |
[13:27:32-090421]  | item 3078/5131 | ms/user 12.44 |
[13:27:38-090421]  | item 3591/5131 | ms/user 12.23 |
[13:27:44-090421]  | item 4104/5131 | ms/user 12.11 |
[13:27:50-090421]  | item 4617/5131 | ms/user 11.97 |
[13:27:56-090421]  | item 5130/5131 | ms/user 11.72 |
[13:27:56-090421]  | training complete | total training time 64.35 s |


In [48]:
sparse_sampler.test()
results = evaluate(slim, sparse_sampler, ["ap@5000"])
collect_results(results)

/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.1413211884342339, 0.08234902229175063)}

In [49]:
slim.save_model("slimHorizontal")

[13:28:22-090421]  Saving SLIM model to slimHorizontal...
[13:28:22-090421]  Model saved!


In [15]:
slim2 = SLIM(l1_reg=.0003, l2_reg=.03)
slim2 = slim2.load_model("slimHorizontal")

[17:52:13-130421]  Loading SLIM model from slimHorizontal...
[17:52:13-130421]  Model loaded!


In [16]:
results = evaluate(slim2, sparse_sampler, ["ap@5000"])
collect_results(results)

[17:52:13-130421]  Sampler must be in valid or test mode. Froced switch to test mode!
/home/jimi/music_fairness/music_fairness/lib/python3.6/site-packages/rectorch/metrics.py:245: RuntimeWarning: invalid value encountered in multiply
  num = pred_scores * ground_truth


{'ap@5000': (0.1413211884342339, 0.08234902229175063)}

In [41]:
sparse_sampler.test()
for _, (data_input, ground_truth) in enumerate(sparse_sampler):
    data_input, ground_truth = prepare_for_prediction(data_input, ground_truth)
    predictions = slim2.predict(*data_input)[0].cpu().numpy()
print(predictions)
print(ground_truth)

print(len(predictions))
print(predictions[0,:])

[[      -inf       -inf       -inf ... 0.04992882 0.4544747  0.00770693]
 [0.00049819 0.00201185       -inf ... 0.         0.         0.00244589]
 [0.07114389       -inf 0.26453635 ... 0.00564663 0.02116286 0.01082972]
 ...
 [0.03638217 0.05174858 0.07925528 ... 0.         0.               -inf]
 [0.10311809 0.07772502 0.25239915 ... 0.00382676 0.15936436       -inf]
 [0.03482109 0.00276908 0.03124328 ... 0.         0.03244206 0.        ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
2804
[      -inf       -inf       -inf ... 0.04992882 0.4544747  0.00770693]


In [14]:
datasetFull = DataProcessing(cfg_data_full).process_and_split()

[15:06:38-020421]  Reading raw data file data_events.csv.
[15:06:42-020421]  Applying filtering.
[15:06:42-020421]  Shuffling data.
[15:06:42-020421]  Calculating splits.
[15:06:42-020421]  Creating validation and test set.
[15:06:42-020421]  Skipped 25464 ratings in validation set.
[15:06:42-020421]  Skipped 24239 ratings in test set.


In [ ]:
"""sparse_sampler_full = SparseDummySampler(datasetFull, mode="train")
sparse_sampler_full.train()
slimFull = SLIM(l1_reg=0.0003 , l2_reg=0.03)
slimFull.train(sparse_sampler_full)
sparse_sampler_full.test()
results = evaluate(slimFull, sparse_sampler_full, ["ap@5000"])
collect_results(results)
slimFull.save_model("slimFull")"""

[18:00:13-020421]  | item 30513/305139 | ms/user 341.08 |
[20:51:57-020421]  | item 61026/305139 | ms/user 337.69 |
[23:45:33-020421]  | item 91539/305139 | ms/user 341.39 |
[02:38:00-030421]  | item 122052/305139 | ms/user 339.08 |
[05:32:15-030421]  | item 152565/305139 | ms/user 342.64 |
[08:28:11-030421]  | item 183078/305139 | ms/user 345.95 |


In [53]:
def get_top_artists(predictions, back_to_user_id,back_to_artist_id, num=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        user_vecs: the feature vector for users
        mf_train: training matrix
        artists_vecs: the feature vector for artists
        back_to_user_id: dictionary to convert back to original user id
        back_to_artist_id: dictionary to convert back to original artist id
        num(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id), ...] of size n.
    """
    top_artists_dict = defaultdict(list)
        # creating dictionary where user id is the key, and the val is a list of tuples of the artist id and 
        # the rating it thinks the user would give it
        
    for user_id in range(len(predictions)):
        
        """pref_vec = mf_train[user_id,:].toarray() # Get the ratings from the training set ratings matrix
        pref_vec = pref_vec.reshape(-1)
        pref_vec =  pref_vec + 1 # Add 1 to everything, so that artists not listened to yet are equal to 1
        pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
        
        rec_vector = user_vecs[user_id,:].dot(artists_vecs.T) # Get dot product of user vector and all artist vectors
        # Scale this recommendation vector between 0 and 1"""
        rec_vector = predictions[user_id,:].copy()
        print(rec_vector)
        
        recommend_vector = rec_vector
        
        product_idx = np.argsort(recommend_vector)[::-1][:num] # Sort the indices of the items into order 
        # of best recommendations
        rec_list = [] # start empty list to store items
        actual_user_id = back_to_user_id[user_id]
        #print(actual_user_id)
        
        for index in product_idx:
            actual_artist_id = back_to_artist_id[index]
            rec_list.append(actual_artist_id)
            
        top_artists_dict[actual_user_id] = rec_list
    print("done")
    return top_artists_dict
    
    

In [21]:
dataset.i2id #dictionary!!

{16122: 0,
 11832: 1,
 1649: 2,
 16037: 3,
 4249: 4,
 9915: 5,
 922: 6,
 15972: 7,
 3241: 8,
 14596: 9,
 12: 10,
 9557: 11,
 13309: 12,
 36385: 13,
 1812: 14,
 7334: 15,
 22404: 16,
 1591: 17,
 13914: 18,
 6306: 19,
 1969: 20,
 1289: 21,
 25362: 22,
 2314: 23,
 9688: 24,
 13535: 25,
 14141: 26,
 16: 27,
 10515: 28,
 3954: 29,
 26153: 30,
 9868: 31,
 35614: 32,
 1703: 33,
 6855: 34,
 12968: 35,
 1958: 36,
 6872: 37,
 4273: 38,
 9670: 39,
 4567: 40,
 924: 41,
 13062: 42,
 8852: 43,
 15101: 44,
 1423: 45,
 3732: 46,
 1628: 47,
 847: 48,
 326: 49,
 9762: 50,
 3044: 51,
 12966: 52,
 11657: 53,
 1883: 54,
 3715: 55,
 29876: 56,
 9699: 57,
 1288: 58,
 15672: 59,
 1526: 60,
 15116: 61,
 557: 62,
 353: 63,
 9852: 64,
 16053: 65,
 13863: 66,
 9772: 67,
 8671: 68,
 10006: 69,
 9334: 70,
 3741: 71,
 2354: 72,
 2511: 73,
 137: 74,
 249: 75,
 9660: 76,
 16946: 77,
 7317: 78,
 36033: 79,
 9969: 80,
 2917: 81,
 9340: 82,
 6171: 83,
 39696: 84,
 3096: 85,
 13522: 86,
 14825: 87,
 10257: 88,
 2325: 89,


In [49]:
id2u =  dict(map(reversed, dataset.u2id.items()))
id2u
id2i = dict(map(reversed, dataset.i2id.items()))
id2u

{0: 1021445,
 1: 1045479,
 2: 1058776,
 3: 1058825,
 4: 1070023,
 5: 1072752,
 6: 1073613,
 7: 2004527,
 8: 2013259,
 9: 2063610,
 10: 2110860,
 11: 2112074,
 12: 2126581,
 13: 2137663,
 14: 2161358,
 15: 2164032,
 16: 2176550,
 17: 2185669,
 18: 2198623,
 19: 2199141,
 20: 2207111,
 21: 2218972,
 22: 2237732,
 23: 2244480,
 24: 2246769,
 25: 2246867,
 26: 2248678,
 27: 2251719,
 28: 2262122,
 29: 2271579,
 30: 2287295,
 31: 2287915,
 32: 2301413,
 33: 2310965,
 34: 2348935,
 35: 2355387,
 36: 2364460,
 37: 2369061,
 38: 2371498,
 39: 2391075,
 40: 2411506,
 41: 2420117,
 42: 2423052,
 43: 2424978,
 44: 2429166,
 45: 2444166,
 46: 2444880,
 47: 2446968,
 48: 2449652,
 49: 2455789,
 50: 2459123,
 51: 2459995,
 52: 2463639,
 53: 2467229,
 54: 2486992,
 55: 2487577,
 56: 2495737,
 57: 2500411,
 58: 2509909,
 59: 2536308,
 60: 2538364,
 61: 2547668,
 62: 2556103,
 63: 2559449,
 64: 2583651,
 65: 2590118,
 66: 2598785,
 67: 2613646,
 68: 2621596,
 69: 2643458,
 70: 2644695,
 71: 2649794,
 7

In [54]:
myDict = get_top_artists(predictions, id2u, id2i)

[      -inf       -inf       -inf ... 0.04992882 0.4544747  0.00770693]
[0.00049819 0.00201185       -inf ... 0.         0.         0.00244589]
[0.07114389       -inf 0.26453635 ... 0.00564663 0.02116286 0.01082972]
[0.01263548 0.01020729 0.07163426 ... 0.0117917  0.00752373 0.01073553]
[0.0175637  0.04654341 0.09109911 ... 0.01896231 0.01048302 0.        ]
[0.00834005 0.00025117 0.05573591 ... 0.         0.00114553 0.00234894]
[0.03557172 0.1170771        -inf ... 0.02002914 0.05648868 0.00671706]
[0.01199605 0.08586507 0.27710032 ... 0.00480297 0.         0.02574964]
[0.07405386 0.02378181 0.3759867  ... 0.01377117 0.00689471 0.0139603 ]
[0.02638558 0.05723609       -inf ... 0.01266015 0.00878339 0.04164375]
[      -inf 0.07499515       -inf ... 0.06857356 0.01479204 0.05773877]
[0.01081714 0.04485501 0.29575753 ... 0.01558227 0.02089094 0.01020647]
[0.0000000e+00 1.2777433e-02          -inf ... 5.5178208e-03 7.4393232e-05
 6.4774272e-03]
[0.00111811 0.00871302 0.10758892 ... 0.00240

[0.         0.00292781 0.01555328 ... 0.         0.         0.00893048]
[0.03902414 0.08919515 0.32884485 ...       -inf 0.00662574 0.02555147]
[0.01337774 0.01998237       -inf ... 0.00561707 0.00748288 0.01197348]
[0.01323506 0.00375052 0.1408922  ... 0.02369724 0.0052724  0.00937868]
[0.01156335 0.02127575       -inf ... 0.00844121 0.00361771 0.00556477]
[0.00707445 0.         0.16427584 ... 0.01408798 0.         0.        ]
[0.         0.00327811       -inf ... 0.0007034  0.0010669  0.01701249]
[0.00414404 0.00036776 0.05014484 ... 0.00877977 0.01067361 0.00830395]
[0.01090445 0.02407233       -inf ... 0.00409901 0.         0.00327953]
[0.07841244 0.10568479 0.07797779 ... 0.01189015 0.03068808 0.0126065 ]
[4.7368538e-02 7.1214348e-02          -inf ... 2.0135656e-02 7.4393232e-05
 4.4237569e-02]
[0.         0.00769619 0.04792337 ... 0.00150825 0.         0.        ]
[0.01673181 0.00543526 0.13969375 ... 0.00150825 0.01598722 0.00053843]
[0.00846659 0.00958161       -inf ... 0.00238

[0.12568784 0.01489553 0.42370692 ... 0.01436309 0.01652793 0.02161307]
[0.12613614 0.07452559       -inf ... 0.00593841 0.00984957 0.04410085]
[0.05126781 0.02125695 0.16189946 ... 0.         0.01156876 0.00555385]
[0.03003934       -inf 0.2157584  ... 0.01514112 0.00783656 0.01490601]
[0.         0.00628903 0.         ... 0.         0.         0.        ]
[0.0160536  0.0019633  0.10002718 ... 0.0124595  0.00026747 0.        ]
[0.00914219 0.04934911 0.11992165 ... 0.00677884 0.01467632 0.01905125]
[0.03890367 0.03519978 0.40528977 ... 0.00946088 0.01848989 0.02333223]
[2.3293085e-02 1.3680848e-01 6.0287779e-03 ... 1.7505736e-03 4.0473972e-02
 8.2424252e-05]
[0.23247106 0.09651194 0.12000648 ... 0.03389591 0.00702705 0.00850156]
[0.         0.0032291  0.03311521 ... 0.01733276 0.02673478 0.00648868]
[0.         0.0094749  0.07460356 ... 0.00124423 0.         0.00662223]
[0.         0.00986802 0.03413836 ... 0.         0.00582445 0.00464107]
[0.01690456 0.09539621 0.10232446 ... 0.01864

[0.13628885 0.19266501 0.15434098 ... 0.03272894 0.16548578 0.03158993]
[0.         0.00184016 0.12455016 ... 0.00437284 0.         0.        ]
[0.04656395 0.12809972 0.2860988  ... 0.03199921 0.0407011  0.02529543]
[0.02217949 0.01257118 0.06330866 ... 0.00052967 0.00640685 0.        ]
[0.         0.01836138 0.05889454 ... 0.00975814 0.00678093 0.        ]
[0.02451946 0.00364572 0.0577539  ... 0.00717442 0.00408755       -inf]
[0.00738715 0.01263523 0.02621421 ... 0.00658839 0.07176671 0.        ]
[0.         0.00292781 0.01281741 ... 0.00418536 0.         0.00399448]
[0.01106814 0.05145863       -inf ... 0.01044208 0.00227016 0.01022406]
[0.08751996 0.05861269 0.26129252 ... 0.04584535       -inf 0.06877314]
[0.0342526  0.00152697 0.17260383 ... 0.00833331 0.00593632 0.01198877]
[0.         0.00604851 0.05690033 ... 0.0200731  0.         0.075608  ]
[0.0531617  0.00602443 0.05084337 ... 0.         0.00250832 0.        ]
[0.297406   0.3507997        -inf ... 0.0505026  0.07607974 0.07

[0.00538403 0.0073438  0.08712052 ... 0.         0.         0.        ]
[0.04595287 0.00036776 0.07538911 ... 0.25932974 0.0073109  0.00053843]
[0.01950094 0.01356113 0.02776074 ... 0.         0.03165706 0.0029228 ]
[4.0343001e-02 2.2670675e-02 8.0050148e-02 ... 2.0011046e-05 2.2378262e-02
 1.7813584e-02]
[0.00214723 0.03285449 0.01827187 ... 0.         0.01873484 0.00167175]
[0.00155135 0.02032407 0.07886134 ... 0.00711138 0.         0.08485261]
[0.05188185 0.04087242 0.3060621  ... 0.00420282 0.01513873 0.00362409]
[0.         0.00267566 0.03053664 ... 0.00150825 0.         0.00801452]
[0.01479071 0.         0.07232685 ... 0.04996479 0.00299357 0.00012606]
[0.01511774 0.         0.0563122  ... 0.         0.00520738 0.01338906]
[0.00780663 0.01965335 0.09014039 ... 0.00368315 0.003193   0.0039506 ]
[0.09342007 0.03782712 0.10831145 ... 0.00255934 0.00171317 0.0291752 ]
[0.00626373 0.04945321 0.3298971  ... 0.01204143 0.01041747 0.02544636]
[0.01119559 0.01320013       -inf ... 0.     

[0.01876242 0.01550277 0.08744741 ... 0.00162539 0.00421157 0.05584031]
[0.13377112 0.07099892       -inf ... 0.0185098  0.04038965 0.03012408]
[0.         0.00548747 0.00959898 ... 0.00014663 0.0402761  0.        ]
[0.00264579 0.01812157 0.00877815 ... 0.00374156 0.00429596 0.01534559]
[0.10830192 0.04988003 0.5427969  ... 0.0374504  0.00620188 0.03468747]
[0.00307802 0.00277741 0.03229198 ... 0.         0.08928508 0.        ]
[0.07213781 0.03273094 0.5531398  ... 0.03270918 0.00195972 0.03088228]
[0.         0.00880156 0.00310555 ... 0.0031136  0.04968273       -inf]
[0.1095387  0.05593143 0.0969026  ... 0.02581006 0.01730616 0.        ]
[0.00704833 0.01006757 0.01834984 ... 0.00391634 0.00401437 0.        ]
[0.00248722 0.01167032 0.03707835 ... 0.         0.05055782 0.01255166]
[0.01262974 0.08522142 0.09016926 ... 0.01077793 0.01538942 0.01587773]
[0.         0.0115239        -inf ... 0.00150825 0.00407295 0.03881445]
[0.08692534 0.06397632 0.1946214  ... 0.00795663 0.06552248 0.01

[0.00400006 0.         0.05216705 ... 0.17079988 0.         0.        ]
[0.15756752 0.03629051       -inf ... 0.01523459 0.01287779 0.00749506]
[0.21134986 0.05110651 0.32910445 ... 0.01454964 0.01218884 0.02205422]
[0.06281509 0.02696718 0.05430136 ... 0.00159617 0.00437232 0.00262957]
[0.17331767 0.2049319  0.20596664 ... 0.04041152 0.03325367 0.07269116]
[0.01062369 0.00641544 0.03200654 ... 0.         0.05268933 0.        ]
[0.06642421 0.0171994  0.09072084 ... 0.00503658 0.00337156 0.01229359]
[0.00602174 0.00282034 0.06467193 ... 0.00091223 0.         0.        ]
[0.00589145 0.00909002 0.04172067 ... 0.00102101 0.00263145 0.00292962]
[0.00208878 0.03424584 0.02165163 ... 0.00027592 0.02090748 0.00053843]
[0.02659188 0.               -inf ... 0.00491052 0.         0.02081788]
[0.00545254 0.         0.02715781 ... 0.         0.00137167 0.00383082]
[0.02189028 0.03089273 0.10133769 ... 0.02834671       -inf 0.01952169]
[0.0130671  0.01896249 0.10355164 ... 0.01721929 0.12016486 0.00

In [55]:
myDict[1021445]

[3424, 227, 114, 360, 1164, 691, 3744, 19349, 10294, 4207]

In [56]:
# Train and test the four algorithms on our data set
low_gap_r_list = []
medium_gap_r_list = []
high_gap_r_list = []
total_gap_r_list= []
#keeps track of the GAPr of each of the algs

alg_recommendations = [] #Keeps track of how many times each artist was recommended

#Will put for loop here with training, but for now we already have predictions
    
num_times_recommended = pd.DataFrame(artist_dist)
num_times_recommended.columns = ['Recommendation Frequency']
num_times_recommended['Recommendation Frequency'] = 0 #resets for each algorithm
    
    
# resets to 0 before calculating for new alg
low_gap_r = 0
medium_gap_r = 0
high_gap_r = 0
    
#keeps track of group size since it is unpredictable since the test set is selected randomly
num_low_users = 0
num_medium_users = 0
num_high_users = 0
    
#keeps track of the mean absolute error of the current alg
low_mae = 0
medium_mae = 0
high_mae = 0
top_artists = myDict #Gets the top N recommendations for each user
    

for user_id, ratings in top_artists.items():
    artist_id_list = [] #user profile size to compute top fraction in GAPr
    for artist_id in ratings:
            
        artist_id_list.append(artist_id)
        num_times_recommended.loc[artist_id] += 1 #increments the number of times the artist was recommended

    gap = sum(artist_dist[artist_id_list]/ num_users) / len(artist_id_list) #fraction in numerator for GAPr
    #print("gap:" ,gap)
    if user_id in low_users.index: #summation of all of the top fractions to compute numerator for GAPr
        low_gap_r += gap
        num_low_users += 1 #increments group size to get the denominator to compute GAPr

    elif user_id in medium_users.index:
        medium_gap_r += gap
        num_medium_users += 1

    elif user_id in high_users.index:
        high_gap_r += gap
        num_high_users += 1
alg_recommendations.append(num_times_recommended)
            
total_gap_r = (low_gap_r + medium_gap_r + high_gap_r) / (num_low_users + num_medium_users + num_high_users)
low_gap_r = low_gap_r / num_low_users #Computing GAPr for each group size for each algorithm
medium_gap_r = medium_gap_r / num_medium_users
high_gap_r = high_gap_r / num_high_users

total_gap_r_list.append(total_gap_r)
low_gap_r_list.append(low_gap_r)
medium_gap_r_list.append(medium_gap_r)
high_gap_r_list.append(high_gap_r)

print("done")

done


In [57]:
#Computes change in GAP for each RecSys alg
delta_gap_low_list = []
delta_gap_medium_list = []
delta_gap_high_list = []
delta_gap_total_list = []

for i in range(len(low_gap_r_list)):
    delta_gap_low = ((low_gap_r_list[i] - low_gap_p) / low_gap_p)
    delta_gap_medium = ((medium_gap_r_list[i] - medium_gap_p) / medium_gap_p)
    delta_gap_high = ((high_gap_r_list[i] - high_gap_p) / high_gap_p)
    delta_gap_total = ((total_gap_r_list[i] - total_gap_p) / total_gap_p)
    
    delta_gap_low_list.append(delta_gap_low)
    delta_gap_medium_list.append(delta_gap_medium)
    delta_gap_high_list.append(delta_gap_high)
    delta_gap_total_list.append(delta_gap_total)
    
print(delta_gap_low_list[0])

print(delta_gap_medium_list[0])

print(delta_gap_high_list[0])

print(delta_gap_total_list[0])

2.663266993343709
2.720945436349978
2.409118205863711
2.5877539912773777
